<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/image_classification_tribeau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [302]:
import os
import json
import requests
import numpy as np
import pandas as pd
from copy import deepcopy
from logging import (
    getLogger,
    StreamHandler,
    DEBUG,
    INFO,
    Formatter,
    config
)
# drive.mount('/content/drive')

In [303]:
# 環境変数読み込み
TARGET_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/accumulated_list_tribeau_eye_until_9000.xlsx'
CHECKED_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/accumulated_list_tribeau_eye_checked.xlsx'
ROOT_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/症例'

#カスタムロガーの設定
with open('/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

In [304]:
# データ読み込み

# ターゲット
target_file_path = TARGET_FILE_PATH
df_target = pd.read_excel(target_file_path)

# 絞り込み用DF
checked_file_path = CHECKED_FILE_PATH
df_checked = pd.read_excel(checked_file_path)

In [305]:
df = df_target[df_target['URL'].isin(df_checked['URL'])]
df = df.fillna('-')
with pd.ExcelWriter("/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/accumulated_list_tribeau_eye_checked_new.xlsx") as writer:
    df.to_excel(writer, sheet_name="Sheet1", index=False)

In [306]:
# 施術名変換

# 変換表生成(施術名)
conversion_dict = {
    "二重埋没 / 目元": "二重/埋没法, Mb",
    "目頭切開 / 目元": "目頭切開, MGs",
    "ヒアルロン酸注入（涙袋） / 目元": "涙袋形成, Nb",
    "クマ治療 / 目元": "クマ取り, Km",
    "その他（目元） / 目元": "その他, Ot",
    "眉下切開 / 目元": "眉下切開, MSk",
    "下まぶたの脂肪取り / 目元": "脂肪取り, Sb",
    "二重切開 / 目元": "二重/目頭切開, Sk",
    "タレ目形成・グラマラスライン（切開） / 目元": "タレ目/目頭切開, Sk",
    "目元修正 / 目元": "その他, Ot",
    "目尻切開 / 目元": "その他, Ot",
    "眼瞼下垂（切開） / 目元": "眼瞼下垂, Gn",
    "脂肪注入（目の下） / 目元": "涙袋形成, Nb",
    "下まぶたたるみ取り / 目元": "シワ取り, Sw",
    "上まぶたたるみ取り / 目元": "シワ取り, Sw",
    "自然癒着法 / 目元": "その他, Ot",
    "ボトックス（タレ目） / 目元": "ボトックス注入, Bt",
    "ヒアルロン酸注入（目の下） / 目元": "シワ取り, Sw",
    "ボトックス（眉間） / 目元": "ボトックス注入, Bt",
    "蒙古ひだ形成 / 目元": "その他, Ot",
    "タレ目形成・グラマラスライン（埋没） / 目元": "タレ目/埋没法, TRMb",
    "前額リフト / 目元": "シワ取り, Sw",
    "眼瞼下垂（非切開） / 目元": "眼瞼下垂, Gn",
    "目尻靭帯移動 / 目元": "その他, Ot",
    "ボトックス（目尻） / 目元": "ボトックス注入, Bt",
    "上まぶたの脂肪取り / 目元": "脂肪取り, Sb",
    "その他 / その他": "その他, Ot"
}

# 変換表生成(経過期間)
period_conversion_dict = {
  "before": "Be",
  "after3Month": "Af3M",
  "after1Week": "Af1W",
  "after1Month": "Af1M",
  "after0Day": "Af0D"
}

# カラム名のリストから "施術_" で始まるカラムをフィルタリング
treatment_columns = [col for col in df.columns if col.startswith("施術_")]

# "施術_" で始まるカラム内からconversion_dictに一致するキーを値に変換
# 一致しないキーがあればNaNに置き換える
for col in treatment_columns:
  df[col] = df[col].apply(lambda x: conversion_dict.get(x, np.nan))

In [307]:
def unique_list(input_list):
    seen = set()
    output = []
    for item in input_list:
        if item not in seen:
            seen.add(item)
            output.append(item)

    return output

def filtered_list(input_list):
  input_list_copy = deepcopy(input_list)
  # "二重"と"タレ目"の施術が二つ行われていた場合
  # "タレ目/" を "タレ目" に変換する処理
  for item in input_list_copy:
        if "二重" in item:  # "二重"が文字列の一部として存在するかを確認
            input_list_copy = [treatment.replace("タレ目/", "タレ目") for treatment in input_list_copy]
            break

  return input_list_copy

def ordered_list(folder_name_list, tag_list):
    # (folder_name, tag) のペアを作成
    paired_list = list(zip(folder_name_list, tag_list))

    # 二重,タレ目,その他を含む要素をそれぞれ抽出
    futae_items = [item for item in paired_list if "二重" in item[0]]
    tareme_items = [item for item in paired_list if "タレ目" in item[0]]
    other_items = [item for item in paired_list if "その他" in item[0]]

    # 二重とタレ目とその他を含まない要素を保存
    left_items = [item for item in paired_list if item not in futae_items and item not in tareme_items and item not in other_items]
    sorted_left_items = sorted(left_items)

    # 要素を順序通りに並び替え
    ordered_items = futae_items + tareme_items + sorted_left_items + other_items

    # folder_name_list と tag_list に再度分割
    ordered_folder_name_list = [item[0] for item in ordered_items]
    ordered_tag_list = [item[1] for item in ordered_items]

    return ordered_folder_name_list, ordered_tag_list

def format_treatment_value(value):
    treatments = value.split(", ")
    folder_name = []
    tag = []
    # 空の文字列をスキップ
    if treatments:
      # 偶数インデックスの要素をfolder_nameに、奇数インデックスの要素をtagに分ける
      folder_name = unique_list(treatments[::2])
      tag = unique_list(treatments[1::2])

      # データ前処理
      filtered_folder_name = filtered_list(folder_name)
      ordered_folder_name, ordered_tag = ordered_list(filtered_folder_name, tag)

    return ('-'.join(ordered_folder_name), ''.join(ordered_tag))


# "施術_"で始まるカラムを選択
treatment_columns = df.filter(like="施術_").columns

# それらのカラムを結合。NaNは空文字列に変換
df['combined'] = df[treatment_columns].apply(lambda row: ''.join([str(x) + ', ' if pd.notna(x) else '' for x in row]), axis=1).str.rstrip(', ')

# combinedカラムに関数を適用してtagカラムを生成
df['folder_name'], df['tag'] = zip(*df['combined'].apply(format_treatment_value))

# combinedカラムを削除
df.drop('combined', axis=1, inplace=True)

# pd.DataFrame(df['folder_name']).head(100)

In [308]:
df

,クリニック名,クリニック住所,URL,ドクター名,メニュー名,費用,副作用・リスク,患者属性,施術_1,施術_2,...,after0Day_5,after0Day_6,after0Day_7,after0Day_8,after0Day_9,after0Day_10,after0Day_11,after0Day_12,folder_name,tag
9,LIBER CLINIC,埼玉県さいたま市大宮区仲町2-9 仲町シロタビルEAST302,https://tribeau.jp/case_reports/31454,矢橋 洋一郎,直後からアイメイクも可能！切らない二重もやっぱりやはし！,モニター価格168000円,腫れ、内出血など,女性,"二重/埋没法, Mb",NaN,...,-,-,-,-,-,-,-,-,二重/埋没法,Mb
12,LIBER CLINIC,埼玉県さいたま市大宮区仲町2-9 仲町シロタビルEAST302,https://tribeau.jp/case_reports/31356,矢橋 洋一郎,目頭切開＆二重！目元整形ならやっぱりやはし！,本文記載,腫れ、内出血、瘢痕など,女性,"二重/埋没法, Mb","目頭切開, MGs",...,-,-,-,-,-,-,-,-,二重/埋没法-目頭切開,MbMGs
13,LIBER CLINIC,埼玉県さいたま市大宮区仲町2-9 仲町シロタビルEAST302,https://tribeau.jp/case_reports/31557,矢橋 洋一郎,左右のバランスも整えるデザイン！目元整形はやっぱりやはし,モニター価格128000円,腫れ、内出血など,女性,"二重/埋没法, Mb",NaN,...,-,-,-,-,-,-,-,-,二重/埋没法,Mb
14,LIBER CLINIC,埼玉県さいたま市大宮区仲町2-9 仲町シロタビルEAST302,https://tribeau.jp/case_reports/31561,矢橋 洋一郎,他院脱脂後の凹みにナノファット注入！,モニター価格68000円＋採取作成料153000円,腫れ、内出血、感染など,女性,"その他, Ot","涙袋形成, Nb",...,-,-,-,-,-,-,-,-,涙袋形成-その他,NbOt
15,LIBER CLINIC,埼玉県さいたま市大宮区仲町2-9 仲町シロタビルEAST302,https://tribeau.jp/case_reports/12899,矢橋 洋一郎,切らない二重によるたるみ治療,"128,000円（モニター価格）",腫れ、内出血など,40代女性,"二重/埋没法, Mb",NaN,...,-,-,-,-,-,-,-,-,二重/埋没法,Mb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8957,麹町皮ふ科・形成外科クリニック,東京都千代田区平河町1-4-5 平和第一ビル地下1階,https://tribeau.jp/case_reports/741,-,目のクマ（経結膜脱脂術）,"198,000",内出血、腫れ、痛み、目の,40代女性,"その他, Ot","クマ取り, Km",...,-,-,-,-,-,-,-,-,クマ取り-シワ取り-その他,KmSwOt
8958,リーダース整形外科,大丘市中区東城路 ２５ ヤングスクウェア６階,https://tribeau.jp/case_reports/736,イ・サンユン院長,二重切開＋目頭切開,250万ウォン,出血、非対称など,20代女性,"二重/目頭切開, Sk","目頭切開, MGs",...,-,-,-,-,-,-,-,-,二重/目頭切開-目頭切開,SkMGs
8959,MIYAフェイスクリニック,大阪府大阪市中央区難波4-1-15 近鉄難波ビル3F,https://tribeau.jp/case_reports/705,宮里 裕,埋没重瞼（3点埋没法）,58000,内出血がでたり、違和感を感じる場合があります。腫れやむくみ、感染症、血腫などが起こる可能性が...,30代女性,"二重/埋没法, Mb",NaN,...,-,-,-,-,-,-,-,-,二重/埋没法,Mb
8963,グランド整形外科,ソウル市江南区新沙洞514-16 アックジョンビル,https://tribeau.jp/case_reports/693,キム・ジュンヒョン,自然癒着二重手術、鼻整形手術、脂肪移植,自然癒着二重手術 200万ウォンから 、鼻整形手術 400万ウォンから、脂肪移植 180万...,手術後、予期せぬ出血や炎症、感染症などの副作用が生じることがあります。,20代女性,"二重/埋没法, Mb","目頭切開, MGs",...,-,-,-,-,-,-,-,-,二重/埋没法-目頭切開,MbMGs


In [309]:
# 画像を保存
def save_image_from_url(url, save_path):
    # URLから画像をバイナリデータとしてダウンロード
    response = requests.get(url, stream=True)
    response.raise_for_status()

    # ファイル名と拡張子を取得
    base_name, ext = os.path.splitext(save_path)

    # 既存のファイル名と重複しないファイル名を生成
    counter = 1
    while os.path.exists(save_path):
        save_path = f"{base_name}_{counter}{ext}"
        counter += 1

    # ダウンロードしたバイナリデータをローカルのファイルに保存
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)


def save_image(index, row):
  root = ROOT_FILE_PATH
  subdirectories = ['before', 'after3Month', 'after1Week', 'after1Month', 'after0Day']

  # パスを組み合わせる
  path = f'/{row["folder_name"]}({row["tag"]})'
  target_path = root + path

  # ディレクトリが存在しない場合は作成
  if not os.path.exists(target_path):
      os.makedirs(target_path)

  # サブディレクトリを作成
  # 抽出したいカラムの基本文字列
  subdirectories = ['before', 'after3Month', 'after1Week', 'after1Month', 'after0Day']

  # データフレームのカラム名の中から、subdirectoriesの文字列を含むカラムを抽出
  columns_to_extract = [col for col in df.columns if any(sub in col for sub in subdirectories)]

  for column in columns_to_extract:
    if not pd.isnull(row[column]) and row[column] != "-":
      column_name = column.split('_')[0]
      sub_path = os.path.join(target_path, column_name)
      if not os.path.exists(sub_path):
          os.makedirs(sub_path)

      #画像を保存
      url = row[column]
      tag_name = row['tag']
      period_name = period_conversion_dict[column_name]
      file_name = f'emEye{index}{tag_name}{period_name}'

      save_path = os.path.join(sub_path, f'{file_name}.jpg')  # 保存先のディレクトリとファイル名を指定
      save_image_from_url(url, save_path)

for index, row in df.iterrows():
  logger.debug(f'進捗: {index + 1}/{len(df)}')
  save_image(index, row)

# :黒い四角_小:画像ネーミンングルール ※命名規則キャメルケースに沿っています
# emEye(No.)(施術名)(ビフォー)(アフター)(経過期間)
# 例：emEye2KmBeAf0D　→ No.2 クマ取り ビフォー アフター施術直後
# 例：emEye3SkBeAf1M　→No.3 目頭切開 ビフォー アフター1ヶ月

ストリーミング出力は最後の 5000 行に切り捨てられました。
2023-10-28 17:12:48,541 [DEBUG] main: 進捗: 3202/7535
2023-10-28 17:12:49,891 [DEBUG] main: 進捗: 3203/7535
2023-10-28 17:12:51,232 [DEBUG] main: 進捗: 3204/7535
2023-10-28 17:12:52,384 [DEBUG] main: 進捗: 3205/7535
2023-10-28 17:12:53,629 [DEBUG] main: 進捗: 3206/7535
2023-10-28 17:12:54,948 [DEBUG] main: 進捗: 3207/7535
2023-10-28 17:12:56,729 [DEBUG] main: 進捗: 3208/7535
2023-10-28 17:12:58,028 [DEBUG] main: 進捗: 3209/7535
2023-10-28 17:12:59,235 [DEBUG] main: 進捗: 3210/7535
2023-10-28 17:13:00,407 [DEBUG] main: 進捗: 3211/7535
2023-10-28 17:13:01,612 [DEBUG] main: 進捗: 3212/7535
2023-10-28 17:13:03,056 [DEBUG] main: 進捗: 3213/7535
2023-10-28 17:13:04,328 [DEBUG] main: 進捗: 3215/7535
2023-10-28 17:13:05,591 [DEBUG] main: 進捗: 3217/7535
2023-10-28 17:13:06,901 [DEBUG] main: 進捗: 3218/7535
2023-10-28 17:13:08,141 [DEBUG] main: 進捗: 3219/7535
2023-10-28 17:13:09,332 [DEBUG] main: 進捗: 3220/7535
2023-10-28 17:13:10,526 [DEBUG] main: 進捗: 3221/7535
2023-10-28 17:13:11,717 [DEBUG] 